# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_text_list = source_text.split('\n')
    target_text_list = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
    
    source_ret = [None] * len(source_text_list)
    target_ret = [None] * len(target_text_list)
    for i, sentence in enumerate(source_text_list):
        source_ret[i] = [source_vocab_to_int[word] for word in sentence.split(' ') if word in source_vocab_to_int]
    for i, sentence in enumerate(target_text_list):
        target_ret[i] = [target_vocab_to_int[word] for word in sentence.split(' ') if word in target_vocab_to_int]
    return source_ret, target_ret

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0


/Users/pranavsharma/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], "input")
    targets = tf.placeholder(tf.int32, [None, None], "targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    target_sequence_length = tf.placeholder(tf.int32,[None], name="target_sequence_length")
    max_sequence_length = tf.reduce_max(target_sequence_length, name='max_sequence_length')
    source_sequence_length = tf.placeholder(tf.int32, [None], name='source_sequence_length')
    
    return (inputs, targets, learning_rate, keep_prob, target_sequence_length, max_sequence_length, source_sequence_length)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/Users/pranavsharma/anaconda/envs/tensorflow/lib/python3.5/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/Users/pranavsharma/anaconda/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/Users/pranavsharma/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/Users/pranavsharma/anaconda/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/Users/pranavsharma/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start\n

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    #return None
    # Create a constant tensor with the 'go id'.
    go_id = tf.constant(target_vocab_to_int['<GO>'], shape=(batch_size,1), dtype=tf.int32)
    # Concatenate the vector without the last word id with the go ids vector
    processed_input = tf.concat([go_id,target_data[:,:-1]],1)
    return processed_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    #return None, None
    embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    #Create RNN Cell
    def create_cell(rnn_size):
        cell = tf.contrib.rnn.LSTMCell(rnn_size)
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob = keep_prob)
        return cell
    cell = tf.contrib.rnn.MultiRNNCell([create_cell(rnn_size) for layer in range(num_layers)])
    rnn_output, rnn_state = tf.nn.dynamic_rnn(cell, embed_input, sequence_length = source_sequence_length, dtype = tf.float32)
    return (rnn_output, rnn_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, encoder_state, output_layer)
    output, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder,maximum_iterations=max_summary_length)
    return output
    #return None



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # Convert the start_ids to be a vector with batch size (the go id repeated batch size times)
    ids = tf.tile([start_of_sequence_id], [batch_size])
    # Create the embedding helper.
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, ids, end_of_sequence_id)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, encoder_state, output_layer)
    output, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder,maximum_iterations=max_target_sequence_length)
    return output
    #return None



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    #return None, None
    def create_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([create_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    embed_input = tf.nn.embedding_lookup(embeddings, dec_input)

    dense_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        Training_BasicDecoderOutput = decoding_layer_train(encoder_state, stacked_lstm, embed_input, target_sequence_length, max_target_sequence_length, dense_layer, keep_prob)
        scope.reuse_variables()
        Inference_BasicDecoderOutput = decoding_layer_infer(encoder_state, stacked_lstm, embeddings, target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], max_target_sequence_length, target_vocab_size, dense_layer, batch_size, keep_prob)
    return Training_BasicDecoderOutput, Inference_BasicDecoderOutput


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    #return None, None
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length, source_vocab_size, enc_embedding_size)
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    Training_BasicDecoderOutput, Inference_BasicDecoderOutput = decoding_layer(processed_input, state,target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    return Training_BasicDecoderOutput, Inference_BasicDecoderOutput 


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.6
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/538 - Train Accuracy: 0.2320, Validation Accuracy: 0.3162, Loss: 5.7413
Epoch   0 Batch    2/538 - Train Accuracy: 0.2516, Validation Accuracy: 0.3157, Loss: 5.5546
Epoch   0 Batch    3/538 - Train Accuracy: 0.2289, Validation Accuracy: 0.3157, Loss: 5.4026
Epoch   0 Batch    4/538 - Train Accuracy: 0.2369, Validation Accuracy: 0.3157, Loss: 5.2011
Epoch   0 Batch    5/538 - Train Accuracy: 0.2626, Validation Accuracy: 0.3157, Loss: 4.9349
Epoch   0 Batch    6/538 - Train Accuracy: 0.2686, Validation Accuracy: 0.3157, Loss: 4.7443
Epoch   0 Batch    7/538 - Train Accuracy: 0.2457, Validation Accuracy: 0.3166, Loss: 4.7029
Epoch   0 Batch    8/538 - Train Accuracy: 0.2557, Validation Accuracy: 0.3272, Loss: 4.6045
Epoch   0 Batch    9/538 - Train Accuracy: 0.2543, Validation Accuracy: 0.3276, Loss: 4.5243
Epoch   0 Batch   10/538 - Train Accuracy: 0.2477, Validation Accuracy: 0.3379, Loss: 4.5511
Epoch   0 Batch   11/538 - Train Accuracy: 0.2693, Validation Accuracy

Epoch   0 Batch   90/538 - Train Accuracy: 0.4369, Validation Accuracy: 0.4711, Loss: 2.4335
Epoch   0 Batch   91/538 - Train Accuracy: 0.4141, Validation Accuracy: 0.4808, Loss: 2.5592
Epoch   0 Batch   92/538 - Train Accuracy: 0.4357, Validation Accuracy: 0.4815, Loss: 2.4971
Epoch   0 Batch   93/538 - Train Accuracy: 0.4158, Validation Accuracy: 0.4764, Loss: 2.5142
Epoch   0 Batch   94/538 - Train Accuracy: 0.4402, Validation Accuracy: 0.4918, Loss: 2.5635
Epoch   0 Batch   95/538 - Train Accuracy: 0.4927, Validation Accuracy: 0.4964, Loss: 2.2818
Epoch   0 Batch   96/538 - Train Accuracy: 0.4624, Validation Accuracy: 0.4801, Loss: 2.3257
Epoch   0 Batch   97/538 - Train Accuracy: 0.4299, Validation Accuracy: 0.4945, Loss: 2.4687
Epoch   0 Batch   98/538 - Train Accuracy: 0.4781, Validation Accuracy: 0.4964, Loss: 2.2858
Epoch   0 Batch   99/538 - Train Accuracy: 0.4373, Validation Accuracy: 0.4927, Loss: 2.4728
Epoch   0 Batch  100/538 - Train Accuracy: 0.4375, Validation Accuracy

Epoch   0 Batch  179/538 - Train Accuracy: 0.4432, Validation Accuracy: 0.4906, Loss: 1.7576
Epoch   0 Batch  180/538 - Train Accuracy: 0.4929, Validation Accuracy: 0.5021, Loss: 1.7067
Epoch   0 Batch  181/538 - Train Accuracy: 0.4375, Validation Accuracy: 0.5004, Loss: 1.7959
Epoch   0 Batch  182/538 - Train Accuracy: 0.4270, Validation Accuracy: 0.4980, Loss: 1.7782
Epoch   0 Batch  183/538 - Train Accuracy: 0.4721, Validation Accuracy: 0.4947, Loss: 1.6850
Epoch   0 Batch  184/538 - Train Accuracy: 0.4760, Validation Accuracy: 0.5004, Loss: 1.6677
Epoch   0 Batch  185/538 - Train Accuracy: 0.4424, Validation Accuracy: 0.4778, Loss: 1.7129
Epoch   0 Batch  186/538 - Train Accuracy: 0.4405, Validation Accuracy: 0.4743, Loss: 1.6883
Epoch   0 Batch  187/538 - Train Accuracy: 0.4901, Validation Accuracy: 0.4954, Loss: 1.6103
Epoch   0 Batch  188/538 - Train Accuracy: 0.4297, Validation Accuracy: 0.4949, Loss: 1.7304
Epoch   0 Batch  189/538 - Train Accuracy: 0.4480, Validation Accuracy

Epoch   0 Batch  268/538 - Train Accuracy: 0.4926, Validation Accuracy: 0.5272, Loss: 1.3456
Epoch   0 Batch  269/538 - Train Accuracy: 0.4623, Validation Accuracy: 0.5215, Loss: 1.4135
Epoch   0 Batch  270/538 - Train Accuracy: 0.4365, Validation Accuracy: 0.4957, Loss: 1.4266
Epoch   0 Batch  271/538 - Train Accuracy: 0.4408, Validation Accuracy: 0.5030, Loss: 1.4264
Epoch   0 Batch  272/538 - Train Accuracy: 0.4533, Validation Accuracy: 0.5273, Loss: 1.4804
Epoch   0 Batch  273/538 - Train Accuracy: 0.4717, Validation Accuracy: 0.5110, Loss: 1.4177
Epoch   0 Batch  274/538 - Train Accuracy: 0.4234, Validation Accuracy: 0.4961, Loss: 1.4452
Epoch   0 Batch  275/538 - Train Accuracy: 0.4650, Validation Accuracy: 0.5144, Loss: 1.4308
Epoch   0 Batch  276/538 - Train Accuracy: 0.4891, Validation Accuracy: 0.5252, Loss: 1.4112
Epoch   0 Batch  277/538 - Train Accuracy: 0.4766, Validation Accuracy: 0.5195, Loss: 1.3839
Epoch   0 Batch  278/538 - Train Accuracy: 0.4656, Validation Accuracy

Epoch   0 Batch  357/538 - Train Accuracy: 0.5152, Validation Accuracy: 0.5376, Loss: 1.2036
Epoch   0 Batch  358/538 - Train Accuracy: 0.5066, Validation Accuracy: 0.5485, Loss: 1.1912
Epoch   0 Batch  359/538 - Train Accuracy: 0.5292, Validation Accuracy: 0.5433, Loss: 1.1442
Epoch   0 Batch  360/538 - Train Accuracy: 0.4992, Validation Accuracy: 0.5442, Loss: 1.2101
Epoch   0 Batch  361/538 - Train Accuracy: 0.5272, Validation Accuracy: 0.5455, Loss: 1.1474
Epoch   0 Batch  362/538 - Train Accuracy: 0.5342, Validation Accuracy: 0.5476, Loss: 1.1109
Epoch   0 Batch  363/538 - Train Accuracy: 0.5242, Validation Accuracy: 0.5467, Loss: 1.1310
Epoch   0 Batch  364/538 - Train Accuracy: 0.4898, Validation Accuracy: 0.5463, Loss: 1.2118
Epoch   0 Batch  365/538 - Train Accuracy: 0.5143, Validation Accuracy: 0.5462, Loss: 1.1658
Epoch   0 Batch  366/538 - Train Accuracy: 0.5217, Validation Accuracy: 0.5424, Loss: 1.1814
Epoch   0 Batch  367/538 - Train Accuracy: 0.5182, Validation Accuracy

...

Epoch   9 Batch  161/538 - Train Accuracy: 0.9334, Validation Accuracy: 0.9329, Loss: 0.0583
Epoch   9 Batch  162/538 - Train Accuracy: 0.9366, Validation Accuracy: 0.9357, Loss: 0.0628
Epoch   9 Batch  163/538 - Train Accuracy: 0.9237, Validation Accuracy: 0.9325, Loss: 0.0708
Epoch   9 Batch  164/538 - Train Accuracy: 0.9430, Validation Accuracy: 0.9339, Loss: 0.0648
Epoch   9 Batch  165/538 - Train Accuracy: 0.9356, Validation Accuracy: 0.9267, Loss: 0.0530
Epoch   9 Batch  166/538 - Train Accuracy: 0.9621, Validation Accuracy: 0.9277, Loss: 0.0531
Epoch   9 Batch  167/538 - Train Accuracy: 0.9304, Validation Accuracy: 0.9249, Loss: 0.0784
Epoch   9 Batch  168/538 - Train Accuracy: 0.9078, Validation Accuracy: 0.9370, Loss: 0.0760
Epoch   9 Batch  169/538 - Train Accuracy: 0.9463, Validation Accuracy: 0.9318, Loss: 0.0557
Epoch   9 Batch  170/538 - Train Accuracy: 0.9235, Validation Accuracy: 0.9370, Loss: 0.0649
Epoch   9 Batch  171/538 - Train Accuracy: 0.9486, Validation Accuracy

Epoch   9 Batch  250/538 - Train Accuracy: 0.9391, Validation Accuracy: 0.9368, Loss: 0.0665
Epoch   9 Batch  251/538 - Train Accuracy: 0.9578, Validation Accuracy: 0.9400, Loss: 0.0586
Epoch   9 Batch  252/538 - Train Accuracy: 0.9422, Validation Accuracy: 0.9453, Loss: 0.0519
Epoch   9 Batch  253/538 - Train Accuracy: 0.9271, Validation Accuracy: 0.9471, Loss: 0.0620
Epoch   9 Batch  254/538 - Train Accuracy: 0.9348, Validation Accuracy: 0.9428, Loss: 0.0604
Epoch   9 Batch  255/538 - Train Accuracy: 0.9389, Validation Accuracy: 0.9357, Loss: 0.0566
Epoch   9 Batch  256/538 - Train Accuracy: 0.9295, Validation Accuracy: 0.9350, Loss: 0.0660
Epoch   9 Batch  257/538 - Train Accuracy: 0.9487, Validation Accuracy: 0.9348, Loss: 0.0591
Epoch   9 Batch  258/538 - Train Accuracy: 0.9485, Validation Accuracy: 0.9345, Loss: 0.0589
Epoch   9 Batch  259/538 - Train Accuracy: 0.9693, Validation Accuracy: 0.9354, Loss: 0.0505
Epoch   9 Batch  260/538 - Train Accuracy: 0.9116, Validation Accuracy

Epoch   9 Batch  339/538 - Train Accuracy: 0.9358, Validation Accuracy: 0.9442, Loss: 0.0610
Epoch   9 Batch  340/538 - Train Accuracy: 0.9299, Validation Accuracy: 0.9437, Loss: 0.0622
Epoch   9 Batch  341/538 - Train Accuracy: 0.9514, Validation Accuracy: 0.9414, Loss: 0.0513
Epoch   9 Batch  342/538 - Train Accuracy: 0.9312, Validation Accuracy: 0.9414, Loss: 0.0594
Epoch   9 Batch  343/538 - Train Accuracy: 0.9512, Validation Accuracy: 0.9371, Loss: 0.0625
Epoch   9 Batch  344/538 - Train Accuracy: 0.9580, Validation Accuracy: 0.9428, Loss: 0.0505
Epoch   9 Batch  345/538 - Train Accuracy: 0.9433, Validation Accuracy: 0.9331, Loss: 0.0561
Epoch   9 Batch  346/538 - Train Accuracy: 0.9200, Validation Accuracy: 0.9263, Loss: 0.0674
Epoch   9 Batch  347/538 - Train Accuracy: 0.9391, Validation Accuracy: 0.9306, Loss: 0.0560
Epoch   9 Batch  348/538 - Train Accuracy: 0.9438, Validation Accuracy: 0.9331, Loss: 0.0550
Epoch   9 Batch  349/538 - Train Accuracy: 0.9611, Validation Accuracy

Epoch   9 Batch  428/538 - Train Accuracy: 0.9613, Validation Accuracy: 0.9324, Loss: 0.0470
Epoch   9 Batch  429/538 - Train Accuracy: 0.9394, Validation Accuracy: 0.9322, Loss: 0.0586
Epoch   9 Batch  430/538 - Train Accuracy: 0.9258, Validation Accuracy: 0.9409, Loss: 0.0552
Epoch   9 Batch  431/538 - Train Accuracy: 0.9365, Validation Accuracy: 0.9403, Loss: 0.0505
Epoch   9 Batch  432/538 - Train Accuracy: 0.9403, Validation Accuracy: 0.9409, Loss: 0.0643
Epoch   9 Batch  433/538 - Train Accuracy: 0.9316, Validation Accuracy: 0.9386, Loss: 0.0755
Epoch   9 Batch  434/538 - Train Accuracy: 0.9471, Validation Accuracy: 0.9437, Loss: 0.0536
Epoch   9 Batch  435/538 - Train Accuracy: 0.9436, Validation Accuracy: 0.9370, Loss: 0.0519
Epoch   9 Batch  436/538 - Train Accuracy: 0.9326, Validation Accuracy: 0.9299, Loss: 0.0653
Epoch   9 Batch  437/538 - Train Accuracy: 0.9482, Validation Accuracy: 0.9313, Loss: 0.0621
Epoch   9 Batch  438/538 - Train Accuracy: 0.9490, Validation Accuracy

Epoch   9 Batch  517/538 - Train Accuracy: 0.9537, Validation Accuracy: 0.9377, Loss: 0.0532
Epoch   9 Batch  518/538 - Train Accuracy: 0.9418, Validation Accuracy: 0.9402, Loss: 0.0638
Epoch   9 Batch  519/538 - Train Accuracy: 0.9466, Validation Accuracy: 0.9384, Loss: 0.0561
Epoch   9 Batch  520/538 - Train Accuracy: 0.9436, Validation Accuracy: 0.9462, Loss: 0.0546
Epoch   9 Batch  521/538 - Train Accuracy: 0.9318, Validation Accuracy: 0.9467, Loss: 0.0650
Epoch   9 Batch  522/538 - Train Accuracy: 0.9484, Validation Accuracy: 0.9494, Loss: 0.0520
Epoch   9 Batch  523/538 - Train Accuracy: 0.9486, Validation Accuracy: 0.9513, Loss: 0.0590
Epoch   9 Batch  524/538 - Train Accuracy: 0.9553, Validation Accuracy: 0.9510, Loss: 0.0589
Epoch   9 Batch  525/538 - Train Accuracy: 0.9615, Validation Accuracy: 0.9402, Loss: 0.0543
Epoch   9 Batch  526/538 - Train Accuracy: 0.9435, Validation Accuracy: 0.9352, Loss: 0.0548
Epoch   9 Batch  527/538 - Train Accuracy: 0.9557, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [20]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    #return None
    ids = []
    for word in sentence.lower().split():
        if word in vocab_to_int:
            ids.append(vocab_to_int[word])
        else:
            ids.append(vocab_to_int['<UNK>'])
    return ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [24]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [75, 151, 13, 39, 125, 71, 162]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [289, 80, 263, 123, 66, 141, 1]
  French Words: il a relaxant en été . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.